In [ ]:
# библиотеки общие
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import plotly.express as px
import numpy as np
from scipy import stats as st
from math import sqrt
from math import factorial
from scipy.stats import shapiro
from scipy.stats import binom, norm
import math
!pip install phik -q
from phik import resources, report
pd.set_option('display.float_format', '{:,.3f}'.format)

# библиотеки линейной регрессии
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# библиотеки логистической регрессии
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# библиотеки для пайплайна
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer

from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer, f1_score, fbeta_score
from sklearn.dummy import DummyClassifier

!pip install imbalanced-learn -q 
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
!pip install shap -q
import shap

# обновление версий библиотек, чтобы современная версия кода работала (Загружаются медленно и не с первого раза. Во всяком случае у меня.)
# !pip install scikit-learn==1.1.0 -q
# !pip install matplotlib==3.5.0 -q

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

from sklearn.model_selection import cross_val_score, cross_val_predict

from sklearn.metrics import ConfusionMatrixDisplay
#np.seterr(invalid='ignore')

from lightgbm import LGBMRegressor
from xgboost import XGBClassifier

from sklearn.impute import KNNImputer
from catboost import Pool, CatBoostRegressor, cv
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from math import ceil
pd.options.display.max_columns = None
import optuna
import joblib

import ast

## Второй проектный месяц - задание Маркетинг
### Описание
Интернет-магазин собирает историю покупателей, проводит рассылки предложений итпланирует будущие продажи. Для оптимизации процессов надо выделить пользователей, которые готовы совершить покупку в ближайшее время.

### Цель
Предсказать вероятность покупки в течение 90 дней

### Задачи
● Изучить данные\
● Разработать полезные признаки\
● Создать модель для классификации пользователей\
● Улучшить модель и максимизировать метрику roc_auc\
● Выполнить тестирование
### Данные

#### apparel-purchases история покупок
● client_id идентификатор пользователя\
● quantity количество товаров в заказе\
● price цена товара\
● category_ids вложенные категории, к которым отнсится товар\
● date дата покупки\
● message_id идентификатор сообщения из рассылки

#### apparel-messages история рекламных рассылок
● bulk_campaign_id идентификатор рекламной кампании\
● client_id идентификатор пользователя\
● message_id идентификатор сообщений\
● event тип действия\
● channel канал рассылки\
● date дата рассылки\
● created_at точное время создания сообщения

#### apparel-target_binary совершит ли клиент покупку в течение следующих 90 дней
● client_id идентификатор пользователя\
● target целевой признак

#### full_campaign_daily_event Агрегация общей базы рассылок по дням и типам событий
● date дата\
● bulk_campaign_id идентификатор рассылки\
● count_event* общее количество каждого события event\
● nunique_event* количество уникальных client_id в каждом событии\
* в именах колонок найдете все типы событий event

#### full_campaign_daily_event_channel Агрегация по дням с учетом событий и каналов рассылки
● date дата\
● bulk_campaign_id идентификатор рассылки\
● count_event*_channel* общее количество каждого события по каналам\
● nunique_event*_channel* количество уникальных client_id по событиям и каналам\
* в именах колонок есть все типы событий event и каналов рассылки channel

#### Ход исследования:
1 Загрузка данных\
2 Предварительная обработка данных\
3 Объединение датафреймов, создание новых признаков и исследовательский анализ данных
4 Обучение моделей машинного обучения\
5 Оценка важности признаков\
6 Общий вывод

## Загрузка данных

In [ ]:
ap = pd.read_csv('/datasets/apparel-purchases.csv', sep=',')

In [ ]:
am = pd.read_csv('/datasets/apparel-messages.csv', sep=',')

In [ ]:
target = pd.read_csv('/datasets/apparel-target_binary.csv', sep=',')

In [ ]:
ce = pd.read_csv('/datasets/full_campaign_daily_event.csv', sep=',')

In [ ]:
cc = pd.read_csv('/datasets/full_campaign_daily_event_channel.csv', sep=',')

In [ ]:
def infohead(a):
    a.info()
    display(a.head())

In [ ]:
infohead(ap)

In [ ]:
ap['date'] = pd.to_datetime(ap['date'], format='%Y-%m-%d')

In [ ]:
infohead(am)

In [ ]:
am['date'] = pd.to_datetime(am['date'], format='%Y-%m-%d')

In [ ]:
am['created_at'] = pd.to_datetime(am['created_at'], format='%Y-%m-%d  %H:%M:%S')

In [ ]:
infohead(target)

In [ ]:
infohead(ce)

In [ ]:
ce['date'] = pd.to_datetime(ce['date'], format='%Y-%m-%d')

In [ ]:
infohead(cc)

In [ ]:
cc['date'] = pd.to_datetime(cc['date'], format='%Y-%m-%d')

### Вывод: Все данные сохранены в 5 датафреймов: ap, am, target, ce, cc.

## Предобработка данных

### Оценим наличие полных дубликатов:

In [ ]:
ap.duplicated().sum()

In [ ]:
am.duplicated().sum()

In [ ]:
target.duplicated().sum()

In [ ]:
ce.duplicated().sum()

In [ ]:
cc.duplicated().sum()

Удалим дубликаты, обнаруженные в датафреймах ap, am:

In [ ]:
ap = ap.drop_duplicates()

In [ ]:
am = am.drop_duplicates()

In [ ]:
ap.duplicated().sum()

In [ ]:
am.duplicated().sum()

### Оценим наличие неявных дубликатов и ошибок в категориальных и бинарных столбцах датафреймов:

#### Оценим датафрейм ap:

In [ ]:
ap['client_id']= ap['client_id'].astype('str')

In [ ]:
ap.select_dtypes(include='object').describe()

In [ ]:
ap['category_ids'].unique()

В столбце 'category_ids' обнаружены пропуски [] и None. Их необходимо будет обработать.

In [ ]:
ap['quantity'].unique()

#### Оценим датафрейм am:

In [ ]:
am['bulk_campaign_id']= am['bulk_campaign_id'].astype('str')

In [ ]:
am['client_id']= am['client_id'].astype('str')

In [ ]:
am.select_dtypes(include='object').describe()

In [ ]:
am.query('client_id=="1515915625516327994"')

In [ ]:
am.query('message_id=="1515915625468060902-4617-6290657dac196"')

In [ ]:
am['event'].unique()

In [ ]:
am['channel'].unique()

#### Оценим датафрейм target

In [ ]:
target['client_id']= target['client_id'].astype('str')

In [ ]:
target['target']= target['target'].astype('str')

In [ ]:
target.select_dtypes(include='object').describe()

Мы видим, что количество уникальных значений столбца client_id в датафрейме target равно количеству уникальных значений столбца client_id в датафрейме aр.

In [ ]:
display(target['target'].value_counts())

Как мы видим, количество target = 1 сильно меньше, чем target = 0.

#### Оценим датафрейм ce

In [ ]:
ce['bulk_campaign_id']= ce['bulk_campaign_id'].astype('str')

In [ ]:
ce.select_dtypes(include='object').describe()

#### Оценим датафрейм cc

In [ ]:
cc['bulk_campaign_id']= cc['bulk_campaign_id'].astype('str')

In [ ]:
cc.select_dtypes(include='object').describe()

### Вывод: проведена предварительная обработка данных: удалены дубликаты, отчмечены аномальные значения. 

## Исследовательский анализ данных, объединение датафреймов и генерация новых признаков

### Объединим датафреймы target с датафреймом ap:

In [ ]:
df = pd.merge(target, ap, how='outer', on=['client_id'])

### Создадим признаки год, месяц, день:

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

### Признак "количество id в товаре"

In [ ]:
df['qnt_id'] = df['category_ids'].str.split(', ').apply(len)

In [ ]:
infohead(df)

In [ ]:
def valuepietrain(a, b):
    display(b)
    display(a[b].value_counts())
    display(a[b].value_counts(normalize=True))
    display(px.pie(a, names=b, title=b))
    display(px.density_heatmap(a, x='target', y=b,
                         color_continuous_scale='Viridis',
                         title=b))

In [ ]:
def desboxshaptrain(a, b):
    display(b)
    display(a[b].describe())
    plt.figure(figsize=(6, 6))
    sb.boxplot(data=a, y=b)
    plt.title(b)
    plt.show()
    plt.figure(figsize=(10, 6))
    sb.histplot(
        data=a,
        x=b,
        hue='target',
        kde=True, 
        alpha=0.6
    )
    plt.twinx()
    sb.boxplot(
        data=a,
        x=b,
        hue='target',
        color='gray',
        width=0.2,
        legend=False
    )
    plt.ylabel('')
    plt.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)
    plt.show()
    
    plt.figure(figsize=(10, 6))
    sb.boxplot(
        data=a,
        x='target',
        y=b,
        hue='target'
    )
    plt.title('Зависимость таргета от фактора')
    plt.legend(title='target')
    plt.show()

In [ ]:
numer0 = ['quantity', 'price', 'year', 'month', 'day']
for i in numer0:
    desboxshaptrain(df, i)

1 Показателям quantity, price - наблюдается большее количество выбросов у таргета = 0, чем у таргета = 1, это м.б. объяснено тем, что значений таргет=0 количествнно больше, чем таргет=1.\
2 Показатели year, day - нет особой разницы между знчениями таргета.\
3 Показатель month - наблюдается разброс таргетов: 0 - 5-10 месяц, 1 - 4-11 месяц.

In [ ]:
cat0 = ['qnt_id', 'year']
for i in cat0:
    valuepietrain(df, i)

1 Количество категорий id в подавляющем большинстве случаев - 4, также встречается 3 и 5.\
2 Данные даны за 3 года: 2024, 2023, 2022. Больше всего покупок было соверешно в 2023 году, при этом самое большое соотношение таргет 1/ таргет 0 наблюдается в 2024 году.

In [ ]:
df.select_dtypes(include='object').describe()

### Создание новых признаков в датафрейме df

#### Признак "общее количество товаров в сообщении по категориям"

In [ ]:
df['total_in_category'] = df.groupby(['message_id', 'category_ids'])['quantity'].transform('sum')

#### Признак "первая категория"

In [ ]:
df = df[df['category_ids'] != '[]']

In [ ]:
df['category_ids'] = df['category_ids'].str.replace("[", "", regex=False)

In [ ]:
df['category_ids'] = df['category_ids'].str.replace("]", "", regex=False)

In [ ]:
df['first_id']=df['category_ids'].str.split(',').str[0]

In [ ]:
df['first_id'] = df['first_id'].str.replace("'", "", regex=False)

In [ ]:
df = df[df['first_id'] != 'None']

In [ ]:
df['first_id'].unique()

In [ ]:
df['first_id'] = df['first_id'].astype(str)

#### Признак "общее количество товаров в сообщении"

In [ ]:
df['total_in_message'] = df.groupby(['message_id'])['total_in_category'].transform('sum')

#### Признак "цена категории товара по сообщениям"

In [ ]:
df['price_categ'] = df.groupby(['message_id', 'category_ids'])['price'].transform('sum')

#### Признак "общая цена товаров в сообщении"

In [ ]:
df['total_price'] = df.groupby(['message_id'])['price'].transform('sum')

#### Признак "количество дней от открытия сообщения до покупки"

In [ ]:
df['days_span'] = df.groupby('message_id')['date'].transform(lambda x: (x.max() - x.min()).days)

In [ ]:
df['days_span'] = df['days_span'] + 1

In [ ]:
df['days_span'].unique()

#### Признак "количество дней активности клиента"

In [ ]:
df['fl_span'] = df.groupby(['client_id'])['days_span'].transform('sum')

#### Признак "средняя цена, которую платит клиент"

In [ ]:
df['mean_price'] = (df.groupby(['client_id'])['total_price'].transform('sum'))/(df['fl_span'])

In [ ]:
df['mean_price'].unique()

In [ ]:
infohead(df)

### Исследовательский анализ датфрейма df:

In [ ]:
numer0 = ['quantity', 'price', 'month', 'day', 'total_in_category', 'total_in_message', 'price_categ', 'total_price', 'days_span', 'fl_span', 'mean_price']
for i in numer0:
    desboxshaptrain(df, i)

Во всех новых показателях - наблюдается большее количество выбросов у таргета = 0, чем у таргета = 1, это м.б. объяснено тем, что значений таргет=0 количествнно больше, чем таргет=1.

In [ ]:
cat0 = ['qnt_id', 'year', 'first_id', ]
for i in cat0:
    valuepietrain(df, i)

Как мы видим, если показатель first_id равен 5, 1 и 3, то таргет равен 0. Также количество строк с таким показателе очень мало. редлагается объединить данные строки в одну:

In [ ]:
df['first_id'] = df['first_id'].replace('3', '9')
df['first_id'] = df['first_id'].replace('1', '9')
df['first_id'] = df['first_id'].replace('5', '9')
df['first_id'].unique()

### Подготовка датафрейма am к объединению с датафреймом df

Удалим лишние столбцы датафрейма df, удалим убликаты:

In [ ]:
df1 = df.drop(['quantity', 'price', 'category_ids', 'day', 'date', 'price_categ'], axis=1)

In [ ]:
df1.duplicated().sum()

In [ ]:
df1 = df1.drop_duplicates()

In [ ]:
df1.query('message_id=="1515915625471150952-10289-63f35a8ca9ac5"')

In [ ]:
df1.select_dtypes(include='object').describe()

Выберем из датафрейма df1 уникальнын комбинации столбцов 'client_id', 'message_id'

In [ ]:
df0 = df1[['client_id', 'message_id']].drop_duplicates()
df0 = df0.drop_duplicates()

In [ ]:
infohead(df0)

In [ ]:
am0 = pd.merge(df0, am, on=['client_id', 'message_id'], how='inner')
infohead(am0)

In [ ]:
am0.query('message_id=="1515915625468060902-4617-6290657dac196"')

### Создадим новые признаки в датафрейме am0

#### Признак "количество оплат на сообщение"

In [ ]:
am0['purchase_count'] = am0.groupby(['message_id'])['event'].transform(lambda x: (x == 'purchase').sum())

#### Признак "непокупочные события"

In [ ]:
am1 = pd.merge(target, am0, on=['client_id'], how='inner')
infohead(am1)

In [ ]:
am1['event'] = am1['event'].astype(str)

In [ ]:
valuepietrain(am1, 'event')

Как мы видим, если в столбце event есть строки 'complain', 'subscribe' или 'unsubscribe', то таргет всегда равен 0. Создадим на это основании новый признак в датафрейме am0:

In [ ]:
am0['event'].unique()

In [ ]:
def event_class(df):
    pur = df['event']
    if pur == 'complain' or pur == 'subscribe' or pur == 'unsubscribe':
        return 0
    else:
        return 1

In [ ]:
am0['class_0'] = am0.apply(event_class, axis=1)

#### Признак "количество событий"

In [ ]:
am0['number_event'] = am0.groupby(['client_id'])['event'].transform('count')

#### Признак "количество дней активности на одно сообщение датафрейма am"

In [ ]:
am0['days_span_am'] = am0.groupby('message_id')['date'].transform(lambda x: (x.max() - x.min()).days)

In [ ]:
am0['days_span_am'] = am0['days_span_am'] + 1

#### Признак "количество дней активности клиента датафрейма am"

In [ ]:
am0['fl_span_am'] = am0.groupby(['client_id'])['days_span_am'].transform('sum')

#### Подготовка датафреймов am0 к объединению:

In [ ]:
am0 = am0.drop(['event', 'created_at', 'date'], axis=1)

In [ ]:
am0.duplicated().sum()

In [ ]:
am0 = am0.drop_duplicates()

In [ ]:
df2 = pd.merge(df1, am0, on=['client_id', 'message_id'], how='left')

In [ ]:
df2['target'].value_counts()

In [ ]:
infohead(df2)

Удалим дубликаты в новом датафрейме:

In [ ]:
df2.duplicated().sum()

In [ ]:
df3 = df2.drop_duplicates()

### Заполнение пропусков датафрейма df3:

In [ ]:
df2['number_event'].unique()

Т.к. нам неизвестно количество событий на месте пропусков в столбце 'number_event', то заполним их 0.

In [ ]:
df3['number_event'] = df3['number_event'].fillna(value=0)

Пропуски в столбцах 'days_span_am' и 'fl_span_am' заполним соответсвующими значениями из столбцов 'days_span' и 'fl_span', т.к. эти столбцы близки, хотя и не аналогичны:

In [ ]:
df3['days_span_am'] = df3['days_span_am'].fillna(value=df3['days_span'])

In [ ]:
df3['fl_span_am'] = df3['fl_span_am'].fillna(value=df3['fl_span'])

Пропуски в столбце 'class_0' заполним 2:

In [ ]:
df3['class_0'] = df3['class_0'].fillna(value=2)

Пропуски в столбце 'channel' заполним 'unknown':

In [ ]:
df3['channel'] = df3['channel'].fillna(value='unknown')

Пропуски в столбце 'purchase_count' заполним занчениями из столбца 'total_in_message', т.к. в норме они должны быть равны:

In [ ]:
df3['purchase_count'] = df3['purchase_count'].fillna(value=df3['total_in_message'])

### Обработка аномальных значений столбцов 'total_in_message' и 'purchase_count'

In [ ]:
result = df3[df3['total_in_message'] < df3['purchase_count']]
display(result)

Наблюдаются строки, в которых значение количества заказанных товаров (столбец 'total_in_message') меньше, чем количество совершенных покупок (столбец 'purchase_count'). Такого не может быть, поэтому заменим такие значения в столбце 'total_in_message' на значение столбца 'purchase_count'.

In [ ]:
df3.loc[df3['total_in_message'] < df3['purchase_count'], 'total_in_message'] = df3['purchase_count']

### Создание новых признаков в датафрейме df3:

#### Признак среднее "среднее количество покупок за период активности"

In [ ]:
df3['mean_purchase'] = (df3.groupby(['client_id'])['purchase_count'].transform('sum'))/(df3['fl_span_am'])

In [ ]:
infohead(df3)

#### Признак "класс - соотношение количества товаров и покупок"

In [ ]:
def event_purchase(df):
    if df['total_in_message'] == df['purchase_count']:
        return 'ok'
    if df['total_in_message'] > df['purchase_count']:
        return 'ne-'
    if df['total_in_message'] < df['purchase_count']:
        return 'ne+'

In [ ]:
df3['class_purchase'] = df3.apply(event_purchase, axis=1)

In [ ]:
df3['class_purchase'].unique()

### Признак "коэффициент соотношения количества товаров и покупок"

In [ ]:
df3['kmp'] = df3['purchase_count']/df3['total_in_message']

In [ ]:
df3.select_dtypes(include='object').describe()

In [ ]:
infohead(df3)

### Исследовательский анализ признаков итогового датафрейма:

In [ ]:
df4 = df3

In [ ]:
infohead(df4)

In [ ]:
cat2 = ['channel', 'class_purchase', 'class_0', 'first_id', 'qnt_id', 'year', 'month']
for i in cat2:
    valuepietrain(df4, i)

In [ ]:
display(df4['target'].value_counts())
display(df4['target'].value_counts(normalize=True))
display(px.pie(df4, names='target', title='target'))

Признак "channel" - если значение признака mobile_push, то соотношение target 1 / target 0 больше, чем в строках с другим значением.

In [ ]:
numer2 = ['total_in_category', 'total_in_message', 'total_price', 'days_span', 'purchase_count', 'days_span_am', 'kmp', 'mean_purchase', 'number_event']
for i in numer2:
    desboxshaptrain(df4, i)

1 Признак 'total_in_message': разброс квантилей 25-75 у таргета 1 незначительно шире, чем у таргета 0.\
2 Признак 'total_in_price': разброс квантилей 25-75 у таргета 1 незначительно уже, чем у таргета 0.\
3 Признак 'kmp': разброс квантилей 25-75 у таргета 1 шире, чем у таргета 0.\
4 Признак 'number_event': разброс квантилей 25-75 у таргета 1 смещен выше, чем у таргета 0.

In [ ]:
infohead(df4)

### Оценка корреляции признаков:

In [ ]:
df5 = df4.drop(columns=['client_id', 'message_id', 'bulk_campaign_id'])

In [ ]:
df5.phik_matrix(interval_cols=['total_in_category', 'total_in_message', 'qnt_id', 'total_price', 'days_span', 'purchase_count', 'days_span_am', 'mp', 'mean_price', 'number_event', 'fl_span_am', 'mean_purchase']).style.background_gradient(cmap='coolwarm')

In [ ]:
df5.drop(columns=['channel', 'class_purchase']).corr(method='spearman').style.background_gradient(cmap='coolwarm')

Как мы видим, значимую корреляцию с target имеют только столбцы: fl_span и number_event.

### Выводы:
1 Проведено объединение датафреймов target, am и ap в единый датафрейм df4, с удалением дубликатов, лишних столбцов и заполнением пропусков.\
2 Созданы новые признаки:\
2.1 year год\
2.2 month месяц\ 

2.3 qnt_id количество id в товаре\
2.4 total_in_category общее количество товаров в сообщении по категориям    \
2.5 first_id - первый id товара \
2.6 total_in_message - общее количество товаров в сообщении \
2.7  total_price - общая цена товаров в сообщении\
2.8  days_span - количество дней от открытия сообщения до покупки\
2.9  fl_span - количество дней активности клиента\   
2.10  mean_price - средняя цена, которую платит клиент\
2.11  purchase_count - количество оплат на сообщение\
2.12  class_0 - непокупочные события\
2.13 number_event - количество событий на клиента\
2.14 days_span_am - количество дней активности на одно сообщение датафрейма am\
2.15 fl_span_am - количество дней активности клиента датафрейма am\
2.16 mean_purchase - среднее количество покупок за период активности\
2.17 class_purchase - класс - соотношение количества товаров и покупок\
2.18 kmp - коэффициент соотношения количества товаров и покупок\
3 Проведен исследовательский анализ нового датафрейма:\
3.1 Показателям quantity, price - наблюдается большее количество выбросов у таргета = 0, чем у таргета = 1, это м.б. объяснено тем, что значений таргет=0 количествнно больше, чем таргет=1.\
3.2 Показатели year, day - нет особой разницы между знчениями таргета.\
3.3 Показатель month - наблюдается разброс таргетов: 0 - 5-10 месяц, 1 - 4-11 месяц.\
3.1 Количество категорий id в подавляющем большинстве случаев - 4, также встречается 3 и 5.\
3.5 Данные даны за 3 года: 2024, 2023, 2022. Больше всего покупок было соверешно в 2023 году, при этом самое большое соотношение таргет 1/ таргет 0 наблюдается в 2024 году.\
3.6 Во всех числовых новых показателях - наблюдается большее количество выбросов у таргета = 0, чем у таргета = 1, это м.б. объяснено тем, что значений таргет=0 количествнно больше, чем таргет=1.\
3.7 Признак "channel" - если значение признака mobile_push, то соотношение target 1 / target 0 больше, чем в строках с другим значением.\
3.8 Признак 'total_in_message': разброс квантилей 25-75 у таргета 1 незначительно шире, чем у таргета 0.\
3.9 Признак 'total_in_price': разброс квантилей 25-75 у таргета 1 незначительно уже, чем у таргета 0.\
3.10 Признак 'kmp': разброс квантилей 25-75 у таргета 1 шире, чем у таргета 0.\
3.11 Признак 'number_event': разброс квантилей 25-75 у таргета 1 смещен выше, чем у таргета 0.\
4 Проведена оценка корреляции между признаками: значимую корреляцию с target имеют только столбцы: fl_span и number_event.

## Обучение моделей

Будет обучено несколько моделей машинного обучения.\
Целевой признак: target. \
Категориальные признаки: 'channel', 'first_id', 'qnt_id'\
Порядковые признаки: 'class_purchase', 'class_0'\
Числовые признаки: 'year', 'month', 'total_in_category', 'total_in_message', 'total_price', 'days_span', 'fl_span', 'mean_price', 'number_event',\
 'purchase_count', 'days_span_am', 'kmp', 'fl_span_am', 'mean_purchase'\
 Метрика: ROC_AUC
 Критерий приемлемости: 0.75-0.8.

In [ ]:
dftrain = df5.drop_duplicates()

In [ ]:
dftrain['first_id'].unique()

In [ ]:
dftrain['qnt_id'].unique()

In [ ]:
infohead(dftrain)

In [ ]:
infohead(dftrain)

In [ ]:
RANDOM_STATE = 42

X_train25, X_test25, y_train25, y_test25 = train_test_split(
    dftrain.drop(columns=['target']),
    dftrain['target'],
    test_size = 0.25, 
    random_state = RANDOM_STATE,
    stratify = dftrain['target'])


label_encoder = LabelEncoder()

# обучите модель и трансформируйте тренировочную выборку 
y_train25 = label_encoder.fit_transform(y_train25)
# трансформируем тестовую выборку
y_test25 = label_encoder.transform(y_test25)

display(X_train25.shape)
display(X_test25.shape)
display(y_train25.shape)
display(y_test25.shape)

# создаём списки с названиями признаков
ohe_columns = ['channel', 'first_id', 'qnt_id']
ord_columns = ['class_purchase', 'class_0']
num_columns = ['year', 'month', 'total_in_category', 'total_in_message',
               'total_price', 'days_span', 'fl_span', 'mean_price', 'number_event',
               'purchase_count', 'days_span_am', 'kmp', 'fl_span_am', 'mean_purchase']  


ohe_pipe = Pipeline(
    [('simpleImputer_ohe', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
     ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore'))
    ])

ord_pipe = Pipeline(
    [('simpleImputer_before_ord', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
     ('ord',  OrdinalEncoder(
                categories=[
                    ['ok', 'ne+', 'ne-', 'unknown'], 
                    [0, 1, 2]
                ], 
                handle_unknown='use_encoded_value', unknown_value=np.nan
            )
        ),
     ('simpleImputer_after_ord', SimpleImputer(missing_values=np.nan, strategy='most_frequent'))
    ]
)



# создаём общий пайплайн для подготовки данных
data_preprocessor = ColumnTransformer(
    [('ohe', ohe_pipe, ohe_columns),
     ('ord', ord_pipe, ord_columns),
     ('num', [MinMaxScaler(), StandardScaler(), RobustScaler()], num_columns)
    ], 
    remainder='passthrough'
)

# создаём итоговый пайплайн: подготовка данных и модель
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

param_grid1 = [
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 40),
        'models__max_features': range(2, 40),
        'models__min_samples_split': range(2, 40),
        'models__criterion': ['gini', 'entropy'],
        'models__class_weight': ['balanced', None],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    }
]
    
    # словарь для модели KNeighborsClassifier() 
param_grid2 = [    
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 40),
        'models__weights': ['uniform', 'distance'],
        'models__metric': ['euclidean', 'manhattan', 'minkowski'],
        'models__leaf_size': range(10, 60),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']   
    }
]
    

In [ ]:
def cv_metrics(y_train25, y_train_pred):
    cv_f1 = f1_score(y_train25, y_train_pred)
    cv_f2 = fbeta_score(y_train25, y_train_pred, beta=2)
    cv_accuracy = accuracy_score(y_train25, y_train_pred)
    cv_recall = recall_score(y_train25, y_train_pred)
    cv_precision = precision_score(y_train25, y_train_pred)
    cv_roc_auc = roc_auc_score(y_train25, y_train_pred, multi_class='ovr')

    display(f"Метрики на кросс-валидации:")
    display(f"F1 score: {cv_f1:.4f}")
    display(f"F2 score: {cv_f2:.4f}")
    display(f"Accuracy: {cv_accuracy:.4f}")
    display(f"Recall: {cv_recall:.4f}")
    display(f"Precision: {cv_precision:.4f}")
    display(f"ROC AUC: {cv_roc_auc:.4f}")

In [ ]:
def metrics(y_test25, y_pred):
    f1 = f1_score(y_test25, y_pred)
    f2 = fbeta_score(y_test25, y_pred, beta=2)
    accuracy = accuracy_score(y_test25, y_pred)
    recall = recall_score(y_test25, y_pred)
    precision = precision_score(y_test25, y_pred)
    roc_auc = roc_auc_score(y_test25, y_pred, multi_class='ovr')
    
    display(f"Метрики по результатам тестовой выборки:")
    display(f"F1 score: {f1:.4f}")
    display(f"F2 score: {f2:.4f}")
    display(f"Accuracy: {accuracy:.4f}")
    display(f"Recall: {recall:.4f}")
    display(f"Precision: {precision:.4f}")
    display(f"ROC AUC: {roc_auc:.4f}")

In [ ]:
f1_scorer = make_scorer(roc_auc_score)

### Обучение модели DecisionTreeClassifier

In [ ]:
randomized_search1 = RandomizedSearchCV(
    pipe_final, 
    param_grid1, 
    cv=6,
    scoring=f1_scorer,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search1.fit(X_train25, y_train25)

y_train_pred1 = cross_val_predict(
    randomized_search1.best_estimator_, 
    X_train25, 
    y_train25, 
    cv=6, 
    method='predict'
)

display('Лучшая модель и её параметры DecisionTreeClassifier:\n\n', randomized_search1.best_estimator_)


cv_metrics(y_train25, y_train_pred1)

best_model1 = randomized_search1.best_estimator_
y_pred1 = best_model1.predict(X_test25)

metrics(y_test25, y_pred1)

cm1 = confusion_matrix(y_test25, y_pred1)

disp1 = ConfusionMatrixDisplay(confusion_matrix=cm1, display_labels=best_model1.classes_)
disp1.plot(cmap=plt.cm.Blues)
plt.title('Матрица ошибок DecisionTreeClassifier')
plt.show()


Как мы видим, модель DecisionTreeClassifier не соответсвует критериям приемлемости.

### Обучение KNeighborsClassifier

In [ ]:
randomized_search2 = RandomizedSearchCV(
    pipe_final, 
    param_grid2, 
    cv=6,
    scoring=f1_scorer,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search2.fit(X_train25, y_train25)

y_train_pred2 = cross_val_predict(
    randomized_search2.best_estimator_, 
    X_train25, 
    y_train25, 
    cv=6, 
    method='predict'
)



display('Лучшая модель и её параметры KNeighborsClassifier:\n\n', randomized_search2.best_estimator_)


cv_metrics(y_train25, y_train_pred2)


best_model2 = randomized_search2.best_estimator_
y_pred2 = best_model2.predict(X_test25)

metrics(y_test25, y_pred2)

cm2 = confusion_matrix(y_test25, y_pred2)

disp2 = ConfusionMatrixDisplay(confusion_matrix=cm2, display_labels=best_model2.classes_)
disp2.plot(cmap=plt.cm.Blues)
plt.title('Матрица ошибок KNeighborsClassifier')
plt.show()

Модель KNeighborsClassifier еще хуже, чем DecisionTreeClassifier

### Обучение модели LogisticRegression

In [ ]:
param_grid3 = [
    # Группа 1: solver='liblinear' → поддерживает 'l1' и 'l2'
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE)],
        'models__penalty': ['l1', 'l2'],
        'models__C': np.logspace(-4, 4, 20),  # лучше лог-шкала для C
        'models__solver': ['liblinear'],
        'models__max_iter': [1200],
        'models__class_weight': ['balanced', None],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    
    # Группа 2: solver='saga' → поддерживает 'l1', 'l2', 'elasticnet'
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE)],
        'models__penalty': ['l1', 'l2', 'elasticnet'],
        'models__C': np.logspace(-4, 4, 20),
        'models__l1_ratio': [0.1, 0.5, 0.9],  # только для elasticnet
        'models__solver': ['saga'],
        'models__max_iter': [1200],
        'models__class_weight': ['balanced', None],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    # Группа 3: solver='lbfgs', 'newton-cg', 'sag' → только 'l2' или None
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE)],
        'models__penalty': ['l2', None],
        'models__C': np.logspace(-4, 4, 20),
        'models__solver': ['lbfgs', 'newton-cg', 'sag'],
        'models__max_iter': [1200],
        'models__class_weight': ['balanced', None],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    }
]

randomized_search3 = RandomizedSearchCV(
    pipe_final, 
    param_grid3, 
    cv=6,
    scoring=f1_scorer,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search3.fit(X_train25, y_train25)

display('Лучшая модель и её параметры LogisticRegression:\n\n', randomized_search3.best_estimator_)

y_train_pred3 = cross_val_predict(
    randomized_search3.best_estimator_, 
    X_train25, 
    y_train25, 
    cv=6, 
    method='predict'
)


cv_metrics(y_train25, y_train_pred3)


best_model3 = randomized_search3.best_estimator_
y_pred3 = best_model3.predict(X_test25)

metrics(y_test25, y_pred3)

cm3 = confusion_matrix(y_test25, y_pred3)

disp3 = ConfusionMatrixDisplay(confusion_matrix=cm3, display_labels=best_model3.classes_)
disp3.plot(cmap=plt.cm.Blues)
plt.title('Матрица ошибок LogisticRegression')
plt.show()

Модель LogisticRegression уступает обоим своим предшественникам.

### Обучение модели LGBMClassifier

In [ ]:
param_grid5 = [    
    {   'models': [LGBMClassifier(random_state=RANDOM_STATE)],
        'models__n_estimators': [125, 250, 370],
        'models__max_depth': range(-1, 16),
        'models__learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
        'models__num_leaves': range(15, 120),
        'models__min_child_samples': range(1, 15),
        'models__min_split_gain': [0.0, 0.1],
        'models__class_weight': ['balanced', None],
        'models__verbose': [-1],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }
]

randomized_search5 = RandomizedSearchCV(
    pipe_final, 
    param_grid5, 
    cv=6,
    scoring=f1_scorer,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search5.fit(X_train25, y_train25)

display('Лучшая модель и её параметры LGBMClassifier:\n\n', randomized_search5.best_estimator_)

y_train_pred5 = cross_val_predict(
    randomized_search5.best_estimator_, 
    X_train25, 
    y_train25, 
    cv=6, 
    method='predict'
)


cv_metrics(y_train25, y_train_pred5)


best_model5 = randomized_search5.best_estimator_
y_pred5 = best_model5.predict(X_test25)

metrics(y_test25, y_pred5)

cm5 = confusion_matrix(y_test25, y_pred5)

disp5 = ConfusionMatrixDisplay(confusion_matrix=cm5, display_labels=best_model5.classes_)
disp5.plot(cmap=plt.cm.Blues)
plt.title('Матрица ошибок LGBMClassifier')
plt.show()

Модель LGBMClassifier соответсвует критериям приемлемости и вырывается в лидеры.

### Обучение модели RandomForestClassifier

In [ ]:
param_grid7 =  {    
        'models': [RandomForestClassifier(random_state=RANDOM_STATE)],
        'models__n_estimators': [125, 250, 375],
        'models__max_depth': [5, 10, 15, 20, 25, None],
        'models__min_samples_split': range(2, 40),
        'models__min_samples_leaf': range(2, 40),
        'models__class_weight': ['balanced', None],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }



randomized_search7 = RandomizedSearchCV(
    pipe_final, 
    param_grid7, 
    cv=6,
    scoring=f1_scorer,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search7.fit(X_train25, y_train25)

display('Лучшая модель и её параметры RandomForestClassifier:\n\n', randomized_search7.best_estimator_)

y_train_pred7 = cross_val_predict(
    randomized_search7.best_estimator_, 
    X_train25, 
    y_train25, 
    cv=6, 
    method='predict'
)


cv_metrics(y_train25, y_train_pred7)

best_model7 = randomized_search7.best_estimator_
y_pred7 = best_model7.predict(X_test25)

metrics(y_test25, y_pred7)

cm7 = confusion_matrix(y_test25, y_pred7)

disp7 = ConfusionMatrixDisplay(confusion_matrix=cm7, display_labels=best_model7.classes_)
disp7.plot(cmap=plt.cm.Blues)
plt.title('Матрица ошибок RandomForestClassifier')
plt.show()

Модель RandomForestClassifier не соответсвует критериям приемлемости.

### Обучение модели CatBoostClassifier

In [ ]:
param_grid6 = [    
    {   'models': [CatBoostClassifier(random_state=RANDOM_STATE, verbose=False)],
        'models__iterations': [125, 250, 500],
        'models__depth': range(3, 10),
        'models__learning_rate': [0.01, 0.05, 0.1],
        'models__l2_leaf_reg': range(1, 10),
        'models__border_count': range(32, 64),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }
]


randomized_search6 = RandomizedSearchCV(
    pipe_final, 
    param_grid6, 
    cv=6,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search6.fit(X_train25, y_train25)

display('Лучшая модель и её параметры CatBoostClassifier:\n\n', randomized_search6.best_estimator_)

y_train_pred6 = cross_val_predict(
    randomized_search6.best_estimator_, 
    X_train25, 
    y_train25, 
    cv=6, 
    method='predict'
)


cv_metrics(y_train25, y_train_pred6)


best_model6 = randomized_search6.best_estimator_
y_pred6 = best_model6.predict(X_test25)

metrics(y_test25, y_pred6)

cm6 = confusion_matrix(y_test25, y_pred6)

disp6 = ConfusionMatrixDisplay(confusion_matrix=cm6, display_labels=best_model6.classes_)
disp6.plot(cmap=plt.cm.Blues)
plt.title('Матрица ошибок CatBoostClassifier')
plt.show()

Модель CatBoostClassifier не соответсвует критериям приемлемости.

### Обучение модели XGBClassifier

In [ ]:
param_grid9 = {    
        'models': [XGBClassifier(random_state=RANDOM_STATE)],
        'models__iterations': [300, 500, 800, 1200],
        'models__depth': range(3, 10),
        'models__learning_rate': [0.03, 0.05, 0.1],
        'models__l2_leaf_reg': range(1, 10),
        'models__scale_pos_weight': [20, 40, 51, 70, 100, 150],
        'models__border_count': range(64, 254),
        'models__random_strength': [0, 1, 2],
        'models__bagging_temperature': [0, 0.5, 1],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }


randomized_search9 = RandomizedSearchCV(
    pipe_final, 
    param_grid9, 
    cv=6,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search9.fit(X_train25, y_train25)

display('Лучшая модель и её параметры GradientBoostingClassifier:\n\n', randomized_search9.best_estimator_)

y_train_pred9 = cross_val_predict(
    randomized_search9.best_estimator_, 
    X_train25, 
    y_train25, 
    cv=6, 
    method='predict'
)


cv_metrics(y_train25, y_train_pred9)

best_model9 = randomized_search9.best_estimator_
y_pred9 = best_model9.predict(X_test25)

metrics(y_test25, y_pred9)

cm9 = confusion_matrix(y_test25, y_pred9)

disp9 = ConfusionMatrixDisplay(confusion_matrix=cm9, display_labels=best_model9.classes_)
disp9.plot(cmap=plt.cm.Blues)
plt.title('Матрица ошибок XGBClassifier')
plt.show()

Модель XGBClassifier не соответсвует критериям приемлемости.

### Обучение модели GradientBoostingClassifier

In [ ]:
param_grid8 = {    
        'models': [GradientBoostingClassifier(random_state=RANDOM_STATE)],
        'models__n_estimators': [50, 100, 200],
        'models__learning_rate': [0.01, 0.1, 0.2],
        'models__max_depth': range(2, 11),
        'models__subsample': [0.8, 1.0],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }


randomized_search8 = RandomizedSearchCV(
    pipe_final, 
    param_grid8, 
    cv=6,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search8.fit(X_train25, y_train25)

display('Лучшая модель и её параметры GradientBoostingClassifier:\n\n', randomized_search8.best_estimator_)

y_train_pred8 = cross_val_predict(
    randomized_search8.best_estimator_, 
    X_train25, 
    y_train25, 
    cv=6, 
    method='predict'
)


cv_metrics(y_train25, y_train_pred8)

best_model8 = randomized_search8.best_estimator_
y_pred8 = best_model8.predict(X_test25)

metrics(y_test25, y_pred8)

cm8 = confusion_matrix(y_test25, y_pred8)

disp8 = ConfusionMatrixDisplay(confusion_matrix=cm8, display_labels=best_model8.classes_)
disp8.plot(cmap=plt.cm.Blues)
plt.title('Матрица ошибок GradientBoostingClassifier')
plt.show()

Модель GradientBoostingClassifier не соответсвует критериям приемлемости.\
Таким образом лучшей моделью является LGBMClassifier.

### Тюнинг лучшей модели LGBMClassifier при помощи Optuna

In [ ]:
data_preprocessor1 = ColumnTransformer(
    [
        ('ohe', ohe_pipe, ohe_columns),
        ('ord', ord_pipe, ord_columns),
        ('num', StandardScaler(), num_columns)
    ], 
    remainder='passthrough'
)

In [ ]:
pipeline = Pipeline(
    [
        ('preprocessor', data_preprocessor1),
        ('lgbm', LGBMClassifier(random_state=42))  # Добавляем random_state
    ]
)

# Функция для оптимизации
def objective(trial):
    # Определяем пространство поиска гиперпараметров
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', -1, 16)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
    num_leaves = trial.suggest_int('num_leaves', 8, 128)
    min_child_samples = trial.suggest_int('min_child_samples', 1, 100)
    min_split_gain = trial.suggest_loguniform('min_split_gain', 1e-8, 1.0)
    class_weight = trial.suggest_categorical('class_weight', ['balanced', None])
    verbose = trial.suggest_categorical('verbose', [-1])  # 0 - без вывода
    
    # Настраиваем параметры LGBM
    pipeline.set_params(
        lgbm__n_estimators=n_estimators,
        lgbm__max_depth=max_depth,
        lgbm__learning_rate=learning_rate,
        lgbm__num_leaves=num_leaves,
        lgbm__min_child_samples=min_child_samples,
        lgbm__min_split_gain=min_split_gain,
        lgbm__class_weight=class_weight,
        lgbm__random_state=42,
        lgbm__verbose=verbose
    )
    
    try:
        # Вычисляем оценку F1
        scores = cross_val_score(pipeline, X_train25, y_train25, cv=5, scoring=f1_scorer)
        return scores.mean()
    except Exception as e:
        print(f"Ошибка при кросс-валидации: {e}")
        return 0.0

# Запускаем оптимизацию
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=100)

display("Лучшие параметры LGBMClassifier+Optuna:")
display(study.best_params)

pipeline.set_params(
    lgbm__n_estimators=study.best_params['n_estimators'],
    lgbm__max_depth=study.best_params['max_depth'],
    lgbm__learning_rate=study.best_params['learning_rate'],
    lgbm__num_leaves=study.best_params['num_leaves'],
    lgbm__min_child_samples=study.best_params['min_child_samples'],
    lgbm__min_split_gain=study.best_params['min_split_gain'],
    lgbm__class_weight=study.best_params['class_weight'],
    lgbm__random_state=42,
    lgbm__verbose=study.best_params['verbose']
)


y_train_pred = cross_val_predict(pipeline, X_train25, y_train25, cv=5)

cv_metrics(y_train25, y_train_pred)


# Обучаем модель
pipeline.fit(X_train25, y_train25)

# Вычисляем F1 на тестовой выборке
y_pred = pipeline.predict(X_test25)
metrics(y_test25, y_pred)

# Матрица ошибок
cm = confusion_matrix(y_test25, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipeline.named_steps['lgbm'].classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title('Матрица ошибок LGBMClassifier + Optuna')
plt.show()

model5 = pipeline

### Вывод: 
1 Проведено обучение нескольких моделей машинного обучения: DecisionTreeClassifier, KNeighborsClassifier, LogisticRegression, LGBMClassifier, RandomForestClassifier, XGBClassifier, GradientBoostingClassifier.\
2 Проведен тюнинг гиперпараметров лучшей модели XGBClassifier, характеристики лучшей модели:

'Лучшие параметры LGBMClassifier+Optuna:'
{'n_estimators': 791,\
 'max_depth': 10,\
 'learning_rate': 0.028467031729880812,\
 'num_leaves': 57,\
 'min_child_samples': 37,\
 'min_split_gain': 0.9362236544376288,\
 'class_weight': 'balanced',\
 'verbose': -1}

'Метрики на кросс-валидации:'\
'F1 score: 0.3413'\
'F2 score: 0.4558'\
'Accuracy: 0.9273'\
'Recall: 0.5871'\
'Precision: 0.2406'\
'ROC AUC: 0.7628'\

'Метрики по результатам тестовой выборки:'\
'F1 score: 0.3455'\
'F2 score: 0.4678'\
'Accuracy: 0.9255'\
'Recall: 0.6124'\
'Precision: 0.2406'\
'ROC AUC: 0.7741'

 

## Оценим важность факторов лучшей модели:

In [ ]:
def shapvalue(pipeline, X_train, X_test):
    preprocessor = pipeline.named_steps['preprocessor']
    model = pipeline.named_steps['lgbm']
    feature_names = preprocessor.get_feature_names_out()

    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    if hasattr(X_train_transformed, 'toarray'):
        X_train_transformed = X_train_transformed.toarray()
    if hasattr(X_test_transformed, 'toarray'):
        X_test_transformed = X_test_transformed.toarray()


    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test_transformed)


    if isinstance(shap_values, list):
        shap_explanation = shap.Explanation(
            values=shap_values[1],
            base_values=explainer.expected_value[1],
            data=X_test_transformed,
            feature_names=feature_names
        )
    else:
        shap_explanation = shap.Explanation(
            values=shap_values,
            base_values=explainer.expected_value,
            data=X_test_transformed,
            feature_names=feature_names
        )

    return shap_explanation

# Строим график
shap_explanation = shapvalue(model5, X_train25, X_test25)
shap.plots.bar(shap_explanation, max_display=17, show=False)
plt.xlabel("Среднее SHAP-значение признака")
plt.ylabel("Признак")
plt.title("График общей значимости признаков")
plt.show()

In [ ]:
shap.plots.beeswarm(shap_explanation, max_display=16, show=False)
plt.xlabel("SHAP-значение признака")
plt.ylabel("Признак")
plt.title("График влияния признаков на предсказание модели")
plt.show()

12 самых важных признаков:\
1 fl_span - количество дней активности клиента (0.38)\
2 mean_price - средняя цена, которую приносит клиент за период активности (0.35) - чем показатель меньше, тем чаще target 1\
3 mean_purchase - среднее количество покупок за период активности (0.27) - чем показатель меньше, тем чаще target 1\
4 total_price - сумма, принесенная клиентом (0.23), чем она меньше, тем чаще target 1\
5 total_in_message - общее количество товаров в сообщении (0.21) - чем показатель меньше, тем чаще target 1\
6 kmp - коэффициент соотношения товаров и покупок (0.21) - чем он больше, тем чаще target 1\
7 year - год (0.2), чем он больше, тем чаще target 1\
8 first_id_4 - если первый id 4 (0.19), тем чаще target 1\
9 number_event - количество сообщений (0.17)\
10 month - месяц (0.16)\
11 days_span_am - количество дней активности на одно сообщение датафрейма am (0.15)\
12 fl_span_am - количество дней активности клиента датафрейма am (0.14)

In [ ]:
shap.plots.waterfall(shap_explanation[4], show=False)
plt.xlabel("SHAP-значение признака")
plt.ylabel("Признак")
plt.title("Вклад каждого признака в модель для сотрудника")
plt.show()

In [ ]:
shap.plots.waterfall(shap_explanation[58], show=False)
plt.xlabel("SHAP-значение признака")
plt.ylabel("Признак")
plt.title("Вклад каждого признака в модель для сотрудника")
plt.show()

In [ ]:
shap.plots.waterfall(shap_explanation[181], show=False)
plt.xlabel("SHAP-значение признака")
plt.ylabel("Признак")
plt.title("Вклад каждого признака в модель для сотрудника")
plt.show()

## Общий вывод:

1 Все данные сохранены в 5 датафреймов: ap, am, target, ce, cc.

2 Проведена предварительная обработка данных: удалены дубликаты, отчмечены аномальные значения. 

3 Проведено объединение датафреймов target, am и ap в единый датафрейм df4, с удалением дубликатов, лишних столбцов и заполнением пропусков.

4 Созданы новые признаки:\
4.1 year год\
4.2 month месяц\  
4.3 qnt_id количество id в товаре\
4.4 total_in_category общее количество товаров в сообщении по категориям    \
4.5 first_id - первый id товара \
4.6 total_in_message - общее количество товаров в сообщении \
4.7  total_price - общая цена товаров в сообщении\
4.8  days_span - количество дней от открытия сообщения до покупки\
4.9  fl_span - количество дней активности клиента\   
4.10  mean_price - средняя цена, которую платит клиент\
4.11  purchase_count - количество оплат на сообщение\
4.12  class_0 - непокупочные события\
4.13 number_event - количество событий на клиента\
4.14 days_span_am - количество дней активности на одно сообщение датафрейма am\
4.15 fl_span_am - количество дней активности клиента датафрейма am\
4.16 mean_purchase - среднее количество покупок за период активности\
4.17 class_purchase - класс - соотношение количества товаров и покупок\
4.18 kmp - коэффициент соотношения количества товаров и покупок

5 Проведен исследовательский анализ нового датафрейма:\
5.1 Показателям quantity, price - наблюдается большее количество выбросов у таргета = 0, чем у таргета = 1, это м.б. объяснено тем, что значений таргет=0 количествнно больше, чем таргет=1.\
5.2 Показатели year, day - нет особой разницы между знчениями таргета.\
5.3 Показатель month - наблюдается разброс таргетов: 0 - 5-10 месяц, 1 - 4-11 месяц.\
5.4 Количество категорий id в подавляющем большинстве случаев - 4, также встречается 3 и 5.\
5.5 Данные даны за 3 года: 2024, 2023, 2022. Больше всего покупок было соверешно в 2023 году, при этом самое большое соотношение таргет 1/ таргет 0 наблюдается в 2024 году.\
5.6 Во всех числовых новых показателях - наблюдается большее количество выбросов у таргета = 0, чем у таргета = 1, это м.б. объяснено тем, что значений таргет=0 количествнно больше, чем таргет=1.\
5.7 Признак "channel" - если значение признака mobile_push, то соотношение target 1 / target 0 больше, чем в строках с другим значением.\
5.8 Признак 'total_in_message': разброс квантилей 25-75 у таргета 1 незначительно шире, чем у таргета 0.\
5.9 Признак 'total_in_price': разброс квантилей 25-75 у таргета 1 незначительно уже, чем у таргета 0.\
5.10 Признак 'kmp': разброс квантилей 25-75 у таргета 1 шире, чем у таргета 0.\
5.11 Признак 'number_event': разброс квантилей 25-75 у таргета 1 смещен выше, чем у таргета 0.

6 Проведена оценка корреляции между признаками: значимую корреляцию с target имеют только столбцы: fl_span и number_event.

7 Проведено обучение нескольких моделей машинного обучения: DecisionTreeClassifier, KNeighborsClassifier, LogisticRegression, LGBMClassifier, RandomForestClassifier, XGBClassifier, GradientBoostingClassifier.

8 Проведен тюнинг гиперпараметров лучшей модели XGBClassifier, характеристики лучшей модели:

'Лучшие параметры LGBMClassifier+Optuna:'
{'n_estimators': 791,\
 'max_depth': 10,\
 'learning_rate': 0.028467031729880812,\
 'num_leaves': 57,\
 'min_child_samples': 37,\
 'min_split_gain': 0.9362236544376288,\
 'class_weight': 'balanced',\
 'verbose': -1}

'Метрики на кросс-валидации:'\
'F1 score: 0.3413'\
'F2 score: 0.4558'\
'Accuracy: 0.9273'\
'Recall: 0.5871'\
'Precision: 0.2406'\
'ROC AUC: 0.7628'

'Метрики по результатам тестовой выборки:'\
'F1 score: 0.3455'\
'F2 score: 0.4678'\
'Accuracy: 0.9255'\
'Recall: 0.6124'\
'Precision: 0.2406'\
'ROC AUC: 0.7741'

9 Топ-12 самых важных признаков:\
9.1 fl_span - количество дней активности клиента (0.38)\
9.2 mean_price - средняя цена, которую приносит клиент за период активности (0.35) - чем показатель меньше, тем чаще target 1\
9.3 mean_purchase - среднее количество покупок за период активности (0.27) - чем показатель меньше, тем чаще target 1\
9.4 total_price - сумма, принесенная клиентом (0.23), чем она меньше, тем чаще target 1\
9.5 total_in_message - общее количество товаров в сообщении (0.21) - чем показатель меньше, тем чаще target 1\
9.6 kmp - коэффициент соотношения товаров и покупок (0.21) - чем он больше, тем чаще target 1\
9.7 year - год (0.2), чем он больше, тем чаще target 1\
9.8 first_id_4 - если первый id 4 (0.19), тем чаще target 1\
9.9 number_event - количество сообщений (0.17)\
9.10 month - месяц (0.16)\
9.11 days_span_am - количество дней активности на одно сообщение датафрейма am (0.15)\
9.12 fl_span_am - количество дней активности клиента датафрейма am (0.14)

10 Рекомендации для бизнеса:\
10.1 Обратить внимание на клиентов, которые покупаеют за меньшие суммы меньшее количество товаров, но чаще.\
10.2 Обратить внимание на клиентов, которые оплачивают заказанные товары.\
10.3 Клиенты возвращаются в определенные месяцы.\
10.4 Делать рассылки через моюильное приложение.